# **Compute neoclassical theory with ripple with only $N_c q$, $\delta$, $\varepsilon$ and $\nu_\star$**

Note: This is a reduced version of a script containing more information

## <font color='red'>**Import modules, loading datas, defining basic functions**</font>

In [ ]:
import ipywidgets as widgets
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import scipy.integrate as integrate
from scipy.misc import derivative
import h5py as h5
import pickle
import time
from scipy.interpolate import InterpolatedUnivariateSpline
import glob

plt.close('all')
plt.rcParams['axes.formatter.limits'] = (-2,2)
plt.rc('font',size=14)
plt.rcParams.update({"axes.grid" : True})

import matplotlib.colors as mcol
import matplotlib.cm as cm

#Define custom colormap
cm1 = mcol.LinearSegmentedColormap.from_list("MyCmapName",["b","r"])

#Defining array from 0 to 2pi for poloidal averaging
res = 300
axe_theta = np.linspace(0,2*np.pi,res)

# %% Définition des paramètres
N             = 16
Eps           = 3.2
inv_rho_star  = 150
a             = inv_rho_star

## <font color='red'>**Defining collisional functions**</font>

In [ ]:
# %% Collisions related functions
def Phi_v(v)   : return (2/(np.sqrt(np.pi))) * integrate.quad(lambda x: np.exp(-x**2),0,v)[0]
def G_v(v)     : return 1/(2*v**2) * (Phi_v(v) - v * derivative(Phi_v,v,dx=1e-3))
def nu_bar_v(v): return 0.75*np.sqrt(2*np.pi) * v**(-3) * (Phi_v(v) - G_v(v))
def nu_bar_u(u): return nu_bar_v(np.sqrt(u))

# %% Regime discrimination functions
def min_ap(x,y): return x*y/(x+y)
def max_ap(x,y): return x+y

## <font color='purple'> **Definition** </font> - Transport coefficients and $\nu_\varphi$

In [ ]:
I=(4 * 1.38 / np.pi)
sp2 = np.sqrt(np.pi/2)

res = 300
axe_theta = np.linspace(0,2*np.pi,res)
ulim=20

def kT_kE_kP_new(DoE, nustar, Nq):
    
    G0, G0_prime, G0_second, G1 = 0, 0, 0, 0
    for th in axe_theta:
        Y = 2
        if DoE > 1e-7: Y = np.abs(np.sin(th))/(DoE*Nq) #Sécurité pour delta-->0
        if Y >= 1:
            G0         +=0
        if Y < 1:
            DoE_eff = DoE * ( np.sqrt(   1 - Y**2    ) - Y * np.arccos(Y) ) 
            G0_prime   += DoE**2/res
            G0_second  += DoE**(0.5)/res
            G1         += 2 * DoE_eff**(1.5) * np.sin(th)**2 /res
    
    K_u       = 0
    K_hat_u   = 0
    K_tilde_u = 0
    
    d       = np.zeros(3)
    d_tilde = np.zeros(3)
    d_hat   = np.zeros(3)
    
    vres    = 100
    for u in np.linspace(0.01,20,vres):
        K_u       = sp2 * np.exp(-u) * u**2 * min_ap( 1, I * nustar*nu_bar_u(u)/np.sqrt(u))
        K_tilde_u = (2/np.pi)**(1.5)*(1/nustar)*np.exp(-u) * u**(5/2)/nu_bar_u(u)* (\
                        (32/9)* G1  \
                        +2*(1/(Nq))*DoE**2 * (1+(np.pi**2 / 8) * nustar*(Nq)**2*nu_bar_u(u)/np.sqrt(u)))
        K_hat_u   = sp2 * Nq * np.exp(-u) * u**2 * (G0 + min_ap( G0_prime, G0_second*I*nustar/(Nq)*nu_bar_u(u)/np.sqrt(u)))
        
        for n in range(3):
            d[n]       += (u-1.5)**n * K_u
            d_tilde[n] += (u-1.5)**n * K_tilde_u 
            d_hat[n]   += (u-1.5)**n * K_hat_u 

    Delta = (d[0] + d_tilde[0])*(d[0] + d_hat[0])-d[0]**2 
            
    k_T = (d[0] * d_tilde[1] - d[1] * d_tilde[0])  /Delta
    k_E = ((d[0] + d_hat[0])*(d[1] + d_tilde[1]) - d[0]*d[1]) /Delta
    k_P = 1 + k_T - k_E
    return k_T, k_E, k_P

def dcoef(DoE, nustar, Nq):

    G0, G0_prime, G0_second, G1 = 0, 0, 0, 0
    for th in axe_theta:
        Y = 2
        if DoE > 1e-7: Y = np.abs(np.sin(th))/(DoE*Nq) #Sécurité pour delta-->0
        if Y >= 1:
#             G0         += DoE**2/res
            G0         += 0
        if Y < 1:
            DoE_eff = DoE * ( np.sqrt(   1 - Y**2    ) - Y * np.arccos(Y) ) 
            G0_prime   += DoE**2/res
            G0_second  += DoE**(0.5)/res
            G1         += 2 * DoE_eff**(1.5) * np.sin(th)**2 /res
    
    K_u       = 0
    K_hat_u   = 0
    K_tilde_u = 0
    
    d       = np.zeros(3)
    d_tilde = np.zeros(3)
    d_hat   = np.zeros(3)
    
    vrange  = 20
    vres    = 100
    for u in np.linspace(0.001,vrange,vres):
        K_u       = sp2 * np.exp(-u) * u**2 * min_ap( 1, I * nustar*nu_bar_u(u)/np.sqrt(u))
        K_tilde_u = (2/np.pi)**(1.5)*(1/nustar)*np.exp(-u) * u**(5/2)/nu_bar_u(u)* (\
                        (32/9)* G1  \
                        +2*(1/(Nq))*DoE**2 * (1+(np.pi**2 / 8) * nustar*(Nq)**2*nu_bar_u(u)/np.sqrt(u)))
        K_hat_u   = sp2 * Nq * np.exp(-u) * u**2 * (G0 + min_ap( G0_prime, G0_second*I*nustar/(Nq)*nu_bar_u(u)/np.sqrt(u)))
        
        for n in range(3):
            d[n]       += (u-1.5)**n * K_u * (vrange/vres)
            d_tilde[n] += (u-1.5)**n * K_tilde_u * (vrange/vres)
            d_hat[n]   += (u-1.5)**n * K_hat_u * (vrange/vres)
    
    return d, d_tilde, d_hat

def form_factors(DoE, nustar, Nq):
    
    G0, G0_prime, G0_second, G1 = 0, 0, 0, 0
    for th in axe_theta:
        Y = 2
        if DoE > 1e-7: Y = np.abs(np.sin(th))/(DoE*Nq) #Sécurité pour delta-->0
        if Y >= 1:
            #G0         += DoE**2/res
            G0         +=0
        if Y < 1:
            DoE_eff = DoE * ( np.sqrt(   1 - Y**2    ) - Y * np.arccos(Y) ) 
            G0_prime   += DoE**2/res
            G0_second  += DoE**(0.5)/res
            G1         += 2 * DoE_eff**(1.5) * np.sin(th)**2 /res
    return G0, G0_prime, G0_second, G1

## <font color='green'> **Postdoc** </font> - Scan $Nq$ - $k_T$, $k_P$, $k_E$

In [ ]:
plt.rc('font',size=20)
delta     = 0.01
eps       = 0.3
DoE       = delta / eps
nustar_ar = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3]
N         = 18
q_ar      = np.linspace(1,10,10)
Nq_ar     = N * q_ar

color_ar = ['r','g','b','darkorange','purple','steelblue','turquoise']

color_ar = plt.cm.jet(np.linspace(0,1,len(nustar_ar)))

fig = plt.figure(figsize=(12,16)); fig.suptitle(r'$\delta$ = %s | $\varepsilon$ = %s | $N$ = %s' % (delta,eps,N),fontsize=26)
ax_kT = fig.add_subplot(311); ax_kT.set_ylabel(r'$k_{V_T}$',fontsize=26)
ax_kE = fig.add_subplot(312); ax_kE.set_ylabel(r'$k_N$',fontsize=26)
ax_kP = fig.add_subplot(313); ax_kP.set_ylabel(r'$k_{V_P}$',fontsize=26)

for inu,nustar in enumerate(nustar_ar):
    kT_ar,kE_ar,kP_ar = (np.array([]) for i in range(3) )
    kT_ar_mod,kE_ar_mod,kP_ar_mod = (np.array([]) for i in range(3) )
    for iNq,Nq in enumerate(Nq_ar):     
        kT_mod,kE_mod,kP_mod = kT_kE_kP_new(DoE, nustar, Nq)
        kT_ar_mod  = np.append(kT_ar_mod, kT_mod)
        kE_ar_mod  = np.append(kE_ar_mod, kE_mod)      
        kP_ar_mod  = np.append(kP_ar_mod, kP_mod)
    
    ax_kT.plot(q_ar, kT_ar_mod,color=color_ar[inu], ls='--', label=r'$\nu_\star = %.1e$' % nustar,lw=4)
    ax_kE.plot(q_ar, kE_ar_mod,color=color_ar[inu], ls='--', label=r'$\nu_\star = %.1e$' % nustar,lw=4)
    ax_kP.plot(q_ar, kP_ar_mod,color=color_ar[inu], ls='--', label=r'$\nu_\star = %.1e$' % nustar,lw=4)
        
for axes in [ax_kT]:
    handles, labels = axes.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axes.legend(by_label.values(), by_label.keys(),ncol=2,fontsize=17)
for axes in [ax_kT,ax_kE,ax_kP]:
    axes.axhline(y=0,c='k',ls='--',lw=3)
    axes.set_xlabel(r'$q$',fontsize=20)
    
#Asymptotic regimes
ax_kT.axhline(y=1.67,c='r',ls='-')
ax_kT.axhline(y=3.54,c='r',ls='-')
ax_kT.axhline(y=0,c='r',ls='-')

ax_kE.axhline(y=1.5,c='r',ls='-')
ax_kE.axhline(y=3.37,c='r',ls='-')

ax_kP.axhline(y=1.17,c='r',ls='-')
ax_kP.axhline(y=-2.37,c='r',ls='-')
ax_kP.axhline(y=-0.5,c='r',ls='-')
    
fig.tight_layout()

## <font color='green'> **Postdoc** </font> - Scan $\nu^\star$ - $k_T$, $k_P$, $k_E$

In [ ]:
import os
plt.rc('font',size=20)
plt.rcParams['axes.formatter.limits'] = (-5,5)
plt.rcParams.update({"axes.grid" : False})

delta  = 0.005
eps       = 0.3
DoE    = delta / eps
nustar_ar = np.logspace(-2, 1, 100)
N         = 18
q_ar      = np.linspace(1,10,100)
Nq_ar     = N * q_ar

print(Nq_ar[-1])
print(nustar_ar[-1])

fig = plt.figure(figsize=(12,6))#; fig.suptitle(r'$\varepsilon$ = %s | $N$ = %s | $q$ = %s' % (eps,N,q),fontsize=26)
ax = fig.add_subplot(111)

kE_map = np.zeros((len(nustar_ar),len(Nq_ar)))

#Check if 'Figures/kE_map.npy' exists
if os.path.isfile('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps)):
    kE_map = np.load('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps))

else:
    for inu,nustar in enumerate(nustar_ar):
        for iNq,Nq in enumerate(Nq_ar):     
            kE_map[inu,iNq] = kT_kE_kP_new(DoE, nustar, Nq)[1]

    print(kE_map)

np.save('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps),kE_map)

p = ax.pcolormesh(q_ar,nustar_ar,kE_map,cmap='RdYlBu_r')
## Add countour lines where values appears
levels = np.linspace(1.53, 3.25, 6)
CS = ax.contour(q_ar,nustar_ar,kE_map, levels, colors='k', origin='lower',linewidths=2)
ax.clabel(CS, levels, inline=True, fontsize=14)
cbar = fig.colorbar(p,ax=ax)

ax.set_yscale('log')

ax.set_xlabel(r'$q$',fontsize=20)
ax.set_ylabel(r'$\nu_\star$',fontsize=20)

## Add a text box with white background and black border
ax.text(0.05, 0.95, r'$\delta$ = %s | $\varepsilon$ = %s | $N$ = %s' % (delta,eps,N), transform=ax.transAxes, fontsize=20,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=1))

fig.tight_layout()

## <font color='green'> **Postdoc** </font> - Map $(\nu^\star, Nq)$ for a given $\delta/\varepsilon$ - $k_T$, $k_P$, $k_E$

In [ ]:
import os
plt.rc('font',size=20)
plt.rcParams['axes.formatter.limits'] = (-5,5)
plt.rcParams.update({"axes.grid" : False})

data = np.loadtxt('data_exp/exp_profs_45333.dat')
data_q = np.loadtxt('data_exp/exp_profs_q_45333.dat')

data_exp = np.vstack((data,data_q))

param = ['rho','nustar1','nustar2','eps','delta','q1','q2']
data_dic  = {}
for ip,p in enumerate(param):
    data_dic[p] = data_exp[ip,:]


delta  = 0.005
eps       = 0.2
DoE    = delta / eps
nustar_ar = np.logspace(-2, 1, 100)
N         = 18
q_ar      = np.linspace(1,10,100)
Nq_ar     = N * q_ar

print(Nq_ar[-1])
print(nustar_ar[-1])

fig = plt.figure(figsize=(8,6))#; fig.suptitle(r'$\\varepsilon$ = %s | $N$ = %s | $q$ = %s' % (eps,N,q),fontsize=26)
ax = fig.add_subplot(111)

kE_map = np.zeros((len(nustar_ar),len(Nq_ar)))

#Check if 'Figures/kE_map.npy' exists
if os.path.isfile('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps)):
    kE_map = np.load('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps))

else:
    for inu,nustar in enumerate(nustar_ar):
        for iNq,Nq in enumerate(Nq_ar):     
            kE_map[inu,iNq] = kT_kE_kP_new(DoE, nustar, Nq)[1]

    print(kE_map)

np.save('Figures/kE_map_delta%s_eps%s.npy' % (delta,eps),kE_map)

p = ax.pcolormesh(q_ar,nustar_ar,kE_map,cmap='RdYlBu_r',rasterized=True)
## Add countour lines where values appears
levels = np.linspace(1.36, 3.25, 6)
levels = [1.3, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0]
CS = ax.contour(q_ar,nustar_ar,kE_map, levels, colors='k', origin='lower',linewidths=2)
ax.clabel(CS, levels, inline=True, fontsize=14)
cbar = fig.colorbar(p,ax=ax)

ax.set_yscale('log')

ax.set_xlabel(r'$q$',fontsize=20)
ax.set_ylabel(r'$\nu_\star$',fontsize=20)

## Add a text box with white background and black border
ax.text(0.05, 0.95, r'$\delta$ = %s | $\varepsilon$ = %s | $N$ = %s' % (delta,eps,N), transform=ax.transAxes, fontsize=20,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=1))

fig.tight_layout()

fig.savefig('Figures/kE_map_delta%s_eps%s.pdf' % (delta,eps), format='pdf', bbox_inches='tight')

## <font color='green'> **Postdoc** </font> - <font color='royalblue'> (Workshop TSVV) </font>  Map $(\nu^\star, Nq)$ for a given $\delta/\varepsilon$ - $k_T$, $k_P$, $k_E$

In [ ]:
import os
plt.rc('font',size=24)
plt.rcParams['axes.formatter.limits'] = (-5,5)
plt.rcParams.update({"axes.grid" : False})

data = np.loadtxt('data_exp/exp_profs_45333.dat')
data_q = np.loadtxt('data_exp/exp_profs_q_45333.dat')

data_exp = np.vstack((data,data_q))

param = ['rho','nustar1','nustar2','eps','delta','q1','q2']
data_dic  = {}
for ip,p in enumerate(param):
    data_dic[p] = data_exp[ip,:]


delta  = 0.01
eps       = 0.2
DoE    = delta / eps
nustar_ar = np.logspace(-1, 2, 100)
N         = 18
q_ar      = np.linspace(2,6,100)
Nq_ar     = N * q_ar

print(Nq_ar[-1])
print(nustar_ar[-1])

fig = plt.figure(figsize=(9,6))#; fig.suptitle(r'$\\varepsilon$ = %s | $N$ = %s | $q$ = %s' % (eps,N,q),fontsize=26)
ax = fig.add_subplot(111)

kE_map = np.zeros((len(nustar_ar),len(Nq_ar)))

#Check if 'Figures/kE_map.npy' exists
if os.path.isfile('Figures/TSVV_kE_map_delta%s_eps%s.npy' % (delta,eps)):
    kE_map = np.load('Figures/TSVV_kE_map_delta%s_eps%s.npy' % (delta,eps))

else:
    for inu,nustar in enumerate(nustar_ar):
        for iNq,Nq in enumerate(Nq_ar):     
            kE_map[inu,iNq] = kT_kE_kP_new(DoE, nustar, Nq)[1]

    print(kE_map)

np.save('Figures/TSVV_kE_map_delta%s_eps%s.npy' % (delta,eps),kE_map)

p = ax.pcolormesh(q_ar,nustar_ar,kE_map,cmap='RdYlBu_r',rasterized=True)
## Add countour lines where values appears
levels = np.linspace(1.36, 3.25, 6)
levels = [1.4, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0]
CS = ax.contour(q_ar,nustar_ar,kE_map, levels, colors='k', origin='lower',linewidths=2)
ax.clabel(CS, levels, inline=True, fontsize=18)
#cbar = fig.colorbar(p,ax=ax)

#ax.set_ylim(0.1,10)

ax.set_yscale('log')

ax.set_xlabel(r'$q$')
ax.set_ylabel(r'$\nu_\star$')

## Add a text box with white background and black border
ax.text(0.05, 0.1, r'$\delta$ = %s | $\varepsilon$ = %s | $N$ = %s' % (delta,eps,N), transform=ax.transAxes, fontsize=20,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=1))

fig.tight_layout()

fig.savefig('Figures/TSVV_kE_map_delta%s_eps%s.png' % (delta,eps), format='png', bbox_inches='tight')

## <font color='green'> **Postdoc** </font> - <font color='red'> Figure pour Xavier </font> - Scan $\nu^\star$ - $k_T$, $k_P$, $k_E$

In [ ]:
plt.rc('font',size=20)
delta_ar  = np.array([0.005,0.01])
eps       = 0.3
DoE_ar    = delta_ar / eps
nustar_ar = np.logspace(-2,1, 40)
N         = 18
q         = 3
Nq        = N * q

color_ar = ['r','b']

#color_ar = plt.cm.winter(np.linspace(0,1,len(delta_ar)))

fig = plt.figure(figsize=(12,8)); fig.suptitle(r'$\varepsilon$ = %s | $N_{coils}$ = %s | $q$ = %s' % (eps,N,q),fontsize=26)

ax_kE = fig.add_subplot(111); ax_kE.set_ylabel(r'$k_E$',fontsize=26)


for iDoE,DoE in enumerate(DoE_ar):     
    kT_ar,kE_ar,kP_ar = (np.array([]) for i in range(3) )
    kT_ar_mod,kE_ar_mod,kP_ar_mod = (np.array([]) for i in range(3) )
    for inu,nustar in enumerate(nustar_ar):
        kT_mod,kE_mod,kP_mod = kT_kE_kP_new(DoE, nustar, Nq)

        kT_ar_mod  = np.append(kT_ar_mod, kT_mod)
        kE_ar_mod  = np.append(kE_ar_mod, kE_mod)      
        kP_ar_mod  = np.append(kP_ar_mod, kP_mod)
    

    ax_kE.semilogx(nustar_ar, kE_ar_mod,color=color_ar[iDoE], ls='-', label=r'$\delta = %.1e$' % (DoE*eps), lw=4)

        
#for axes in [ax_kT]:
for axes in [ax_kE]:
    handles, labels = axes.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axes.legend(by_label.values(), by_label.keys(),ncol=2,fontsize=25)
for axes in [ax_kE]:
    axes.axhline(y=0,c='k',ls='--',lw=3)
    axes.set_xlabel(r'$\nu_\star$',fontsize=25)
    
#Asymptotic regimes
ax_kE.axhline(y=1.5,c='g',ls='--',lw=2)
ax_kE.axhline(y=3.37,c='g',ls='--',lw=2)


    
fig.tight_layout()